# CSC294-082 Project (transformer generating part)
#### Author: Xinyi Ji

In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
import pandas as pd
import torch.utils.data as data
import pickle as pkl

### Data preprocessing

In [2]:
checkpt = torch.load('transformer_model_new.ckpt')

In [3]:
df = pd.read_csv('transformer_data_new.csv')
df.columns = range(256+1024)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1240,1241,1242,1243,1244,1245,1246,1247,1248,1249,1250,1251,1252,1253,1254,1255,1256,1257,1258,1259,1260,1261,1262,1263,1264,1265,1266,1267,1268,1269,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
0,0,0,0,10,4,0,0,1,8,0,6,1,0,0,0,0,0,0,0,0,0,2,0,1,0,0,4,0,0,0,1,0,0,0,0,0,0,0,0,0,...,4392,7418,3153,3115,1443,369,802,3078,3029,4466,1327,2226,6839,8106,6384,2773,907,3067,6770,7190,2796,3741,5594,5173,1965,1459,3544,5079,1076,7592,2686,35,684,6332,2248,2410,6438,5232,4340,3732
1,0,0,0,12,1,8,0,2,10,1,0,0,0,1,0,0,0,0,0,0,0,10,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,...,5348,3623,4011,8135,3104,5908,3940,7692,4905,7557,5127,5896,7110,7207,443,3162,5485,3460,487,4431,39,1414,1121,2716,1830,5256,6160,5740,5740,1378,165,6997,513,5888,715,7221,1477,1718,6809,7624
2,0,0,0,10,11,2,0,2,10,3,0,0,0,0,0,2,0,0,0,0,0,0,8,0,0,0,3,2,0,0,1,0,0,0,0,0,0,0,0,0,...,2412,5539,4720,3537,8118,8013,5092,6220,986,98,6977,6682,6029,4191,4636,7381,4720,5117,7024,7541,7557,2826,4921,672,7247,2132,2389,7232,6865,2706,7549,7539,7562,2655,6077,1851,5687,3320,6326,3814
3,0,0,0,10,15,0,0,2,9,0,0,3,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,1,0,0,0,0,2,0,0,0,...,3433,1107,4013,5119,3028,5468,6997,2793,4569,8160,7131,7123,4735,348,5997,4521,3740,4264,3901,4742,6441,2285,1343,3924,421,7011,423,2764,3477,421,5608,6583,3044,5886,3821,684,117,4036,5671,5066
4,0,0,0,10,0,9,0,2,10,0,0,2,1,0,0,0,1,1,0,0,0,2,0,1,0,0,1,1,0,0,0,5,0,0,0,1,0,0,0,0,...,2654,7520,5405,568,635,6776,3161,7582,410,5107,7229,4605,5914,6660,4809,2411,2684,4236,7716,4150,2817,2496,1789,7018,4355,6600,906,3773,1824,4698,7239,1732,2817,6614,6321,164,5731,4232,5537,7389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3370,0,0,0,9,8,0,0,1,11,3,2,4,0,0,0,1,1,1,0,0,0,0,11,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,...,49,7143,61,7304,5364,2705,2975,1105,1613,459,8111,3401,5390,5652,1035,5322,2854,2765,7188,7415,6735,4255,3541,4007,5232,4194,4796,1824,6082,4011,4353,3528,3576,7910,255,4291,8135,3997,1062,5628
3371,0,0,0,9,0,8,0,1,14,0,0,4,0,1,0,0,0,1,0,0,0,11,0,4,0,0,0,1,0,0,2,1,0,0,0,0,1,0,0,0,...,4017,785,3158,255,6559,4262,3231,7795,773,8002,4584,746,5970,1846,5780,4786,5210,5734,6936,1520,2672,6135,601,3288,8091,2152,3077,1319,686,4670,7059,2894,48,2950,5364,255,3083,4451,3590,6800
3372,0,0,0,10,0,10,0,1,9,0,0,0,1,0,0,0,1,1,0,0,0,1,0,2,0,0,0,1,0,0,0,2,0,0,0,1,0,0,0,0,...,4005,1958,1430,2070,907,2986,5906,6198,7523,7858,6931,1263,5120,5993,3571,304,1941,6580,2115,2639,1822,7989,7491,2650,4575,951,6082,6148,2136,3528,2496,5415,4005,4824,4147,6639,8025,2533,6422,4835
3373,0,0,0,9,10,0,0,1,8,2,0,2,1,0,0,0,1,2,0,1,0,8,0,5,0,0,0,0,0,0,0,1,0,0,0,2,0,0,1,0,...,8023,3851,7190,4383,7484,3959,7230,4698,5275,2553,2260,3490,4237,7798,6307,2513,2777,4233,3684,6950,6141,4478,7545,916,7520,3577,5048,1075,1204,450,5997,2557,7755,649,3635,3433,1944,6135,4950,4312


In [4]:
txt_col = range(256)
img_col = range(256,1024+256)
df_txt = df[txt_col]
df_img = df[img_col]
txt_array = df_txt.values
img_array = df_img.values
all_data = df.values
all_data = torch.from_numpy(all_data)

In [5]:
all_data.shape[0]

3375

In [6]:
def split_train_test(data, train_fraq=0.8): 
 n_samples = data.shape[0] 
 data_train = data[:int(n_samples * train_fraq)] 
 data_test = data[int(n_samples * train_fraq):] 
 return data_train, data_test 

In [7]:
all_data = split_train_test(all_data)

In [8]:
all_data[0].shape

torch.Size([2700, 1280])

In [9]:
all_data[1].shape

torch.Size([675, 1280])

In [10]:
train_X = all_data[0][:, :256]
train_y = all_data[0][:,256:]
test_X =  all_data[1][:, :256]
test_y = all_data[1][:,256:]

In [11]:
with open('./train_img_token1.pkl','rb') as f: first_token = pkl.load(f)
with open('./train_img_token1_test.pkl','rb') as f: first_token_test = pkl.load(f)


In [12]:
start_token = torch.tensor(first_token) # [2700]
start_token_test = torch.tensor(first_token_test) # [675]

In [13]:
start_token_test.shape

torch.Size([675])

In [14]:
start_token_test.shape

torch.Size([675])

In [15]:
def dataloader(dataset, **kwargs): 
  if 'shuffle' not in kwargs: 
    kwargs['shuffle'] = False 
  if 'drop_last' not in kwargs: 
    kwargs['drop_last'] = True 
  if 'batch_size' not in kwargs: 
    kwargs['batch_size'] = 32 
  if 'num_workers' not in kwargs: 
    kwargs['num_workers'] = 8 
    kwargs['batch_size'] = min(kwargs['batch_size'], len(dataset)) 
  return data.DataLoader(dataset, **kwargs) 

In [16]:
# Train data is a tensor
batch_size = 10
num_workers = 8
train_dataset = data.TensorDataset(train_X, train_y, start_token) 
test_dataset = data.TensorDataset(test_X, test_y, start_token_test) 
train_dataloader = dataloader(train_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True) 
test_dataloader = dataloader(test_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True) 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [17]:
# for batch_idx, (x, y, z) in enumerate(train_dataloader):
#   print(x)
#   print(x.shape)
#   print(y)
#   print(y.shape)
#   print(z)
#   print(z.shape)
#   print("----")

###  Transformer Model:

In [18]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        # Get number of training examples
        N = query.shape[0]

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)  # (N, value_len, heads, head_dim)
        keys = self.keys(keys)  # (N, key_len, heads, head_dim)
        queries = self.queries(query)  # (N, query_len, heads, heads_dim)

        # Einsum does matrix mult. for query*keys for each training example
        # with every other training example, don't be confused by einsum
        # it's just how I like doing matrix multiplication & bmm

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # queries shape: (N, query_len, heads, heads_dim),
        # keys shape: (N, key_len, heads, heads_dim)
        # energy: (N, heads, query_len, key_len)

        # Mask padded indices so their weights become 0
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # Normalize energy values similarly to seq2seq + attention
        # so that they sum to 1. Also divide by scaling factor for
        # better stability
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        # attention shape: (N, heads, query_len, key_len)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # out after matrix multiply: (N, query_len, heads, head_dim), then
        # we reshape and flatten the last two dimensions.

        out = self.fc_out(out)
        # Linear layer doesn't modify the shape, final shape will be
        # (N, query_len, embed_size)

        return out

In [19]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

In [20]:
class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length = 256,
    ):

        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        out = self.dropout(
            (self.word_embedding(x) + self.position_embedding(positions))
        )

        # In the Encoder the query, key, value are all the same, it's in the
        # decoder this will change. This might look a bit odd in this case.
        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out

In [21]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out

In [22]:
class Decoder(nn.Module):
    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length = 1024,
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device),
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device),
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device),
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device),
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, row_mask,col_mask,conv_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        x = self.layers[0](x, enc_out, enc_out, src_mask, row_mask)
        x = self.layers[1](x, enc_out, enc_out, src_mask, col_mask)
        x = self.layers[2](x, enc_out, enc_out, src_mask, row_mask)
        x = self.layers[3](x, enc_out, enc_out, src_mask, row_mask)
        x = self.layers[4](x, enc_out, enc_out, src_mask, conv_mask)
        res = x[:, -1, :]  # [batch_size, num_class]
        out = self.fc_out(x)

        return [out, res]

In [23]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=512,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0,
        device="gpu",
        max_length=1024,
    ):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            256,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            1024,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)

    def make_row_mask(self, trg):
        N, trg_len = trg.shape

        # trg_len = 32 * 32 = 1024
        assert (
            1024 == trg_len
        ), "The size of image vector is 1024"

        y_axis = torch.range(0, 1023).expand(1024,1024)
        x_axis = torch.transpose(torch.range(0, 1023).expand(1024,1024), 0, 1)
        trg_mask = ((y_axis<x_axis).long() + (y_axis+32>x_axis).long() -1).bool()

        trg_mask = trg_mask.expand(N, 1, trg_len, trg_len)

        return trg_mask.to(self.device)
    
    def make_col_mask(self, trg):
        N, trg_len = trg.shape
        # trg_len = 32 * 32 = 1024

        assert (
            1024 == trg_len
        ), "The size of image vector is 1024"

        trg_mask = torch.zeros(1024, 1024)
        for i in range(32):
          cur_dia = torch.ones((1, 1024-32*i))[0]
          cur_mask = torch.diag(cur_dia,-32*i)
          trg_mask = trg_mask + cur_mask

        trg_mask = trg_mask.expand(N, 1, trg_len, trg_len)

        return trg_mask.to(self.device)
    
    def make_conv_mask(self, trg):
      # I am not sure how to deal with this(I searched online without finding the result),hence I will use row mask instead as they have similar performance
        N, trg_len = trg.shape
        # trg_len = 32 * 32 = 1024
        assert (
            1024 == trg_len
        ), "The size of image vector is 1024"

        y_axis = torch.range(0, 1023).expand(1024,1024)
        x_axis = torch.transpose(torch.range(0, 1023).expand(1024,1024), 0, 1)
        trg_mask = ((y_axis<x_axis).long() + (y_axis+32>x_axis).long() -1).bool()

        trg_mask = trg_mask.expand(N, 1, trg_len, trg_len)

        return trg_mask.to(self.device)

    def forward(self, src, trg):
        # src_mask = self.make_src_mask(src)
        # trg_mask = self.make_trg_mask(trg)

        src_mask = self.make_src_mask(src)
        row_mask = self.make_row_mask(trg)
        col_mask = self.make_col_mask(trg)
        conv_mask = self.make_conv_mask(trg)

        enc_src = self.encoder(src, src_mask)
        out, res = self.decoder(trg, enc_src, src_mask, row_mask,col_mask,conv_mask)
        return [out, res]

### dVAE model

In [24]:
# ! pip install DALL-E

In [25]:

# import io
# import os, sys
# import requests
# import PIL

# import torch
# import torchvision.transforms as T
# import torchvision.transforms.functional as TF

# from dall_e          import map_pixels, unmap_pixels, load_model
# from IPython.display import display, display_markdown

# target_image_size = 256

# def download_image(url):
#     resp = requests.get(url)
#     resp.raise_for_status()
#     return PIL.Image.open(io.BytesIO(resp.content))

# def preprocess(img):
#     s = min(img.size)
    
#     if s < target_image_size:
#         raise ValueError(f'min dim for image {s} < {target_image_size}')
        
#     r = target_image_size / s
#     s = (round(r * img.size[1]), round(r * img.size[0]))
#     img = TF.resize(img, s, interpolation=PIL.Image.LANCZOS)
#     img = TF.center_crop(img, output_size=2 * [target_image_size])
#     img = torch.unsqueeze(T.ToTensor()(img), 0)
#     return map_pixels(img)

In [26]:
# # This can be changed to a GPU, e.g. 'cuda:0'.
# dev = torch.device('cpu')

# # For faster load times, download these files locally and use the local paths instead.
# enc = load_model("https://cdn.openai.com/dall-e/encoder.pkl", dev)
# dec = load_model("https://cdn.openai.com/dall-e/decoder.pkl", dev)

In [27]:
# x = preprocess(download_image('https://assets.bwbx.io/images/users/iqjWHBFdfxIU/iKIWgaiJUtss/v2/1000x-1.jpg'))
# display_markdown('Original image:')
# display(T.ToPILImage(mode='RGB')(x[0]))

In [28]:
# import torch.nn.functional as F

# z_logits = enc(x)
# z = torch.argmax(z_logits, axis=1)
# print(z.shape)
# z = F.one_hot(z, num_classes=enc.vocab_size).permute(0, 3, 1, 2).float()

# x_stats = dec(z).float()
# x_rec = unmap_pixels(torch.sigmoid(x_stats[:, :3]))
# x_rec = T.ToPILImage(mode='RGB')(x_rec[0])

# display_markdown('Reconstructed image:')
# display(x_rec)

In [29]:
# z.shape

### Generating part

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def make_src_mask(src):
  src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
  # (N, 1, 1, src_len)
  return src_mask.to(device)

def make_row_mask(trg):
  N, trg_len = trg.shape
  # trg_len = 32 * 32 = 1024
  assert (
      1024 == trg_len
  ), "The size of image vector is 1024"

  y_axis = torch.range(0, 1023).expand(1024,1024)
  x_axis = torch.transpose(torch.range(0, 1023).expand(1024,1024), 0, 1)
  trg_mask = ((y_axis<x_axis).long() + (y_axis+32>x_axis).long() -1).bool()

  trg_mask = trg_mask.expand(N, 1, trg_len, trg_len)

  return trg_mask.to(device)
    
def make_col_mask(trg):
  N, trg_len = trg.shape
  # trg_len = 32 * 32 = 1024

  assert (
      1024 == trg_len
  ), "The size of image vector is 1024"

  trg_mask = torch.zeros(1024, 1024)
  for i in range(32):
    cur_dia = torch.ones((1, 1024-32*i))[0]
    cur_mask = torch.diag(cur_dia,-32*i)
    trg_mask = trg_mask + cur_mask
    trg_mask = trg_mask.expand(N, 1, trg_len, trg_len)

    return trg_mask.to(device)
    
def make_conv_mask(trg):
  # I am not sure how to deal with this(I searched online without finding the result),hence I will use row mask instead as they have similar performance
  N, trg_len = trg.shape
  # trg_len = 32 * 32 = 1024
  assert (
      1024 == trg_len
  ), "The size of image vector is 1024"

  y_axis = torch.range(0, 1023).expand(1024,1024)
  x_axis = torch.transpose(torch.range(0, 1023).expand(1024,1024), 0, 1)
  trg_mask = ((y_axis<x_axis).long() + (y_axis+32>x_axis).long() -1).bool()
  trg_mask = trg_mask.expand(N, 1, trg_len, trg_len)

  return trg_mask.to(device)

In [31]:
def greedy_decode(model, src, src_mask, max_len, start_token, device):
    """
      Args:
        src: [bs, 1024]
        start_token : [bs]
        max_len : [1]
    """
    memory = model.encoder(src, src_mask)  # forward(self, x, mask)
    out_curr = torch.ones(src.shape[0], max_len).long().to(device)  # bs * 1024
    out_curr[: , 0] = start_token.long() # bs * 1024

    row_mask = make_row_mask(out_curr)
    col_mask = make_col_mask(out_curr)
    conv_mask = make_conv_mask(out_curr)

    for i in range(max_len - 1):
      out, res = model.decoder(out_curr, memory, src_mask, row_mask,col_mask,conv_mask)
      prob = out[:, i] # [bs, d_x]
      _, next_img = torch.max(prob, dim = 1) # [bs]
      out_curr[:, i + 1] = next_img.long() 
    #print(res.shape)
    return res 

In [32]:
# src_pad_idx = 0
# trg_pad_idx = 0
# src_vocab_size = 49
# trg_vocab_size = 8192
# model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(device)

# model.load_state_dict(checkpt["model"])
# model.eval()
# #hidden_out = []
# i = 0
# for batch_idx, (x, y, z) in enumerate(train_dataloader):
#   x = x.to(device) 
#   y = y.to(device) 
#   z = z.to(device) #[batch_size]
#   src_mask = make_src_mask(x)
    
#   hidden_out = greedy_decode(model, x, src_mask, max_len=1024, start_token=z, device=device)
#   print(hidden_out)
#   if i == 0:
#     df = pd.DataFrame(hidden_out.cpu().detach().numpy(), columns = range(512))
#   if i > 2:
#     break
#   else:
#     df = pd.concat([df, pd.DataFrame(hidden_out.cpu().detach().numpy(), columns = range(512))])
#   i = i + 1
#   hidden_out = 0
#   print(i)
# #print(hidden_out)
# #print(hidden_out.shape)

In [33]:
# df

In [34]:
# df.to_csv('training_data_mlp.csv', index=False)

In [35]:
src_pad_idx = 0
trg_pad_idx = 0
src_vocab_size = 49
trg_vocab_size = 8192
model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(device)

model.load_state_dict(checkpt["model"])
model.eval()
i = 0
for batch_idx, (x, y, z) in enumerate(test_dataloader):
  x = x.to(device) 
  y = y.to(device) 
  z = z.to(device) #[batch_size]
  src_mask = make_src_mask(x)
    
  hidden_out_test = greedy_decode(model, x, src_mask, 
                        max_len=1024, start_token=z, device=device)
  if i == 0:
    df = pd.DataFrame(hidden_out_test.cpu().detach().numpy(), columns = range(512))
  else:
    df = pd.concat([df, pd.DataFrame(hidden_out_test.cpu().detach().numpy(), columns = range(512))])
  i = i + 1
  hidden_out_test = 0
  print(i)



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [st

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67


In [38]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,1.863567,-0.570675,2.186903,0.342062,-0.107246,0.026681,0.329385,0.449422,-0.157975,-0.506781,1.508040,1.084914,0.211265,-0.355420,0.714799,-0.461057,-0.444154,2.013007,-1.132530,0.230622,0.229829,-0.165674,-0.921479,-0.175453,0.773221,0.279488,-0.031898,0.755713,0.076252,0.617354,0.733818,-2.205909,-0.604884,0.488294,-0.365742,-1.374590,-0.226334,-0.768291,0.393575,-1.290212,...,0.297538,-0.303403,0.092897,1.617078,0.779390,1.589893,0.802914,-0.220412,-0.031898,1.562983,-1.005122,-0.793130,-0.255766,1.318296,3.131931,0.422514,-1.055240,-0.809156,0.265202,0.549371,-0.638051,0.010624,1.043703,-1.142635,-0.435704,-0.647465,1.125153,-0.381220,-0.808859,-1.078182,-0.268686,-0.045226,2.371692,-0.903736,-0.096707,0.261661,-0.851519,-0.376124,0.448598,1.668019
1,1.335494,0.292490,-0.508044,-1.153627,0.540390,0.206045,0.010990,1.304706,-0.538541,-0.326502,0.302313,-0.651378,-0.202800,0.629772,0.371249,-0.324261,-0.694094,1.677536,-0.547544,0.898258,1.244230,0.617386,-1.555147,-0.425234,-1.793179,1.522953,-0.544739,-0.809806,-0.276967,-0.151111,1.508566,-0.490677,1.034442,1.041839,0.125563,-0.308793,1.344633,0.932389,0.024786,-1.020249,...,0.560303,-0.123026,0.046601,0.086926,0.313804,1.956427,-0.487532,0.445130,0.125909,0.819535,1.801975,-0.118825,0.874371,0.240556,1.920498,0.419769,-2.114433,-2.155768,0.125470,0.187012,-1.206990,0.362536,0.322836,-0.747835,0.894047,0.555354,0.223310,-0.460713,-0.382474,0.101025,-0.493817,0.112523,2.793901,-1.576281,-0.597879,-0.123535,-1.407694,0.727233,1.154358,2.425046
2,1.869617,0.700056,2.544178,0.335212,-0.729777,0.127041,1.372651,0.003738,-0.035011,-1.872553,1.099311,0.607663,1.070125,-0.059764,1.031530,1.875414,1.254386,0.853875,-0.940863,-1.121300,0.213144,-1.401598,-0.830637,-0.025231,1.458889,-2.836923,-0.802549,-0.451039,0.043068,0.727391,-1.346132,-0.900911,-0.206616,-0.895753,1.074682,0.239226,-2.445189,-1.311834,0.493376,-1.026084,...,-0.472762,0.214473,-0.528650,2.397090,0.982042,-0.260087,2.482381,-0.535913,0.106498,1.189679,-0.565402,0.006602,-1.950040,1.668866,2.401486,0.296126,0.689231,0.094611,0.587073,0.298768,-0.182701,0.813359,1.474480,0.927866,-0.360115,0.994289,-0.047324,0.160352,-0.043486,-1.417518,-0.851041,0.571906,0.744820,-0.290792,0.470355,2.459215,1.162291,0.174274,0.524155,0.025221
3,1.143558,0.006077,-0.493171,-0.519366,0.064887,-1.035058,-0.447165,2.139800,-0.654380,1.408648,-0.295009,-1.574678,-0.112276,0.692985,-0.731981,2.130375,0.183835,1.539065,-1.162314,-0.045224,0.751372,-0.336167,-0.307350,0.677160,-0.459027,1.729837,0.670097,2.193301,-0.676917,-0.216017,1.584823,0.219403,1.432169,0.287089,0.117677,-2.135802,1.692822,1.476736,-0.051562,0.907624,...,-0.549393,0.791569,0.363926,-1.198233,-0.629785,-0.029873,0.277115,0.774908,1.128809,-0.413325,1.020223,-0.655029,1.203703,0.729719,1.683257,0.050777,-2.632540,-1.959673,0.375993,1.167294,-0.688254,-0.123560,1.446461,0.459580,2.010222,1.371670,-1.813241,-0.265259,-0.451632,-0.485660,-0.531378,0.604779,2.078222,-0.015009,-0.979966,0.980506,0.472184,0.562645,0.800046,1.888708
4,0.222054,0.008657,0.506482,-1.876514,-0.050197,-1.145855,-0.636825,0.563957,-0.243259,0.880066,-0.557536,-0.555836,0.019160,1.014540,1.054555,-1.270604,-1.107479,-1.718342,-1.980413,1.656512,-0.064170,0.727024,-2.266407,-0.113691,-1.051660,1.033901,-0.374226,-0.470933,-1.076191,-0.067301,1.947644,-0.905874,-1.796928,0.027091,-1.989592,1.939876,0.944803,-0.178005,-0.065498,2.330902,...,0.195956,0.445528,0.020233,1.734710,-0.323329,1.589604,-2.185665,0.826422,-0.743552,-1.798128,-0.043457,-1.250349,1.300195,-1.079261,0.769380,0.065627,-1.560810,-0.967591,-0.256781,0.001851,1.117204,-0.638498,-0.636039,-0.027110,0.430437,-0.890558,-0.066978,0.123404,-0.044269,0.721552,0.246

In [39]:
df.to_csv('test_data_mlp.csv', index=False)

Reconstructed image from image vector generated from transformer:

In [36]:
# z = hidden_out.reshape(10, 32, 32)[2][None,...]
# z = F.one_hot(z, num_classes=enc.vocab_size).permute(0, 3, 1, 2).float()

# dec = dec.to(device)
# x_stats = dec(z).float()
# x_rec = unmap_pixels(torch.sigmoid(x_stats[:, :3]))
# x_rec = T.ToPILImage(mode='RGB')(x_rec[0])

# display_markdown('Reconstructed image:')
# display(x_rec)

Original Image:


In [37]:
# z = y.reshape(10, 32, 32)[2][None,...]
# z = F.one_hot(z, num_classes=enc.vocab_size).permute(0, 3, 1, 2).float()

# dec = dec.to(device)
# x_stats = dec(z).float()
# x_rec = unmap_pixels(torch.sigmoid(x_stats[:, :3]))
# x_rec = T.ToPILImage(mode='RGB')(x_rec[0])

# display_markdown('Reconstructed image:')
# display(x_rec)

Although work, but bad performance.